# Argument File Generator

# Torque Bias

## RECONSTRUCTION
Maniobra de referencia 

In [12]:
def get_maneuver():
    manv= input("Maniobra a calcular: \n")#NSSK0812
    return manv

In [13]:
manv= get_maneuver()

Maniobra a calcular: 
NSSK0812


In [14]:
def get_manRef():
    manv_ref= input("Maniobra de referencia: \n")#NSSK0811
    return manv_ref

In [15]:
manv_ref=get_manRef()

Maniobra de referencia: 
NSSK0811


In [32]:
def get_file():
    file = open(fr'V:\mx3\OASYS\Archivos_Mexsat3_2025\MNVR_Report\mx3_{manv_ref}_REC_Duty.rpt', 'r') #referencia
    content = file.read()
    file.close()
    print(content)
    return content

In [33]:
content= get_file()



                     Maneuver Service Report

            Version: OASYS v10.3.3 (September 26, 2012), Kratos Integral Systems International
        Description: 
               Date: 2025/10/14 17:05:50
               Path: /V=/mx3/OASYS/Archivos_Mexsat3_2025/MNVR_Report/mx3_NSSK0811_REC_Duty.rpt
         Spacecraft: [mx3] Mexsat 3
          Ephemeris: /V=/mx3/OASYS/Archivos_Mexsat3_2025/MNVR_Ephemeris/mx3_NSSK0811_REC_pTUpdate.eph
            Equinox: True of Date FK5
           Attitude: AttitudeReference

        Maneuver[s]:         NSSK896
             Status:         Success
               Type:  Reconstruction
    Command Antenna:            None
       Thruster Cfg:      DV_EHT_ODD
        Thruster[s]:           EHT13
                               EHT15

        Firing Mode:      Continuous
Thrust Profile Type:      Continuous
           Ignition:      2025/10/14
                        11:33:51.000 UTC.YmdHMs3
           Centroid:      2025/10/14
                        11

In [34]:
import re

def find_data():
    pattern_epoch="(\d{4})/(\d{2})/(\d{2})\s+(\d{2}):(\d{2}):(\d{2}).(\d{3})"
    pattern_ignition= r"Ignition:\s+"+ pattern_epoch
    pattern_findedIgnition= re.findall(pattern_ignition ,content)
    pattern_cutoff= r"Cutoff:\s+"+ pattern_epoch
    pattern_findedCutoff= re.findall(pattern_cutoff ,content)
    print(type(pattern_findedIgnition))
    print(type(pattern_findedCutoff))
    print(pattern_findedIgnition)
    print(pattern_findedCutoff)
    return pattern_findedIgnition, pattern_findedCutoff

In [35]:
[Ignition, Cutoff]=find_data()

<class 'list'>
<class 'list'>
[('2025', '10', '14', '11', '33', '51', '000')]
[('2025', '10', '14', '12', '20', '59', '003')]


## Get telemetry with library TLM_EPOCH

In [36]:
from TLM_EPOCH import DatoTLM
from datetime import datetime

def get_torque(n):
    datos_torque=DatoTLM("mx3","mx3_rt1@izopsfep3","ACS_ATC_TOTAL_TRQ_"+str(n))
    datos_torque.tipo="completo"
    datos_torque.inicio_TLM=datetime(int(Ignition[0][0]),int(Ignition[0][1]),int(Ignition[0][2]),int(Ignition[0][3]),int(Ignition[0][4]),int(Ignition[0][5]))
    datos_torque.final_TLM=datetime(int(Cutoff[0][0]),int(Cutoff[0][1]),int(Cutoff[0][2]),int(Cutoff[0][3]),int(Cutoff[0][4]),int(Cutoff[0][5]))
    datos_torque.obtener_TLM()
    return datos_torque.xs

In [ ]:
#datos_torque=get_torque(1)
#print(datos_torque)
#print(id(datos_torque))
#print(id(get_torque(3)))

In [37]:
def average_torque(datos_torque):
    #print(id(datos_torque))
    list_addition= sum(datos_torque)
    #print(list_sum)

    list_len= len(datos_torque)
    #print(list_len)

    average= list_addition / list_len
    return average

In [ ]:
#average_torque(datos_torque)

In [38]:
average_torques=[average_torque(get_torque(i)) for i in range(1,4)]

Mnemonico Valido
Obtencion de telemetria exitosa :)
Mnemonico Valido
Obtencion de telemetria exitosa :)
Mnemonico Valido
Obtencion de telemetria exitosa :)


In [39]:
average_torques

[0.0015112709446428551, -0.001275171920053749, 0.005914713588734078]

## Get SKMTool to get the historic data (Pandas)

In [13]:
import pandas as pd

In [87]:
#df= pd.read_excel('SKMTool_dummy.xlsm', sheet_name= 'Historico') This way is incompatible with this pandas ver
df= pd.read_excel('SKMTool_dummy.xlsm', 'Historico')
print(df)
#print(df[['Maniobra', 'Torque 1', 'Torque 2', 'Torque 3']])

     Maniobra  Torque 1  Torque 2  Torque 3 Unnamed: 4       Unnamed: 5  \
0     EWSK260  0.000800  0.026800 -0.000600        NaN              NaN   
1    NSSK0261 -0.133300 -0.008300 -0.026800        NaN              NaN   
2    EWSK0262  0.005000 -0.006000 -0.001000        NaN              NaN   
3    NSSK0263 -0.108300 -0.007200 -0.016600        NaN              NaN   
4    EWSK0264  0.000800  0.026800  0.000600        NaN              NaN   
5    NSSK0265 -0.103300 -0.007200 -0.018600        NaN              NaN   
6    NSSK0266 -0.098200  0.005400  0.049700        NaN              NaN   
7    EWSK0267 -0.001000  0.025300 -0.000300        NaN              NaN   
8    NSSK0268 -0.123000  0.003400  0.064700        NaN              NaN   
9    EWSK0269 -0.001000  0.021300 -0.000800        NaN              NaN   
10   NSSK0270 -0.129000  0.005900  0.062500        NaN              NaN   
11   EWSK0271  0.000500  0.022900 -0.000700        NaN              NaN   
12   NSSK0272 -0.134000  

In [90]:
import pandas as pd

def get_requested_torques(manv_ref):
    df_historic= pd.read_excel('SKMTool_dummy.xlsm', 'Historico')
    requested_torques= df_historic[df.Maniobra== manv_ref]
    requested_torques[['Maniobra', 'Torque 1', 'Torque 2', 'Torque 3']]
    #print(df)
    return df_historic, requested_torques[['Maniobra', 'Torque 1', 'Torque 2', 'Torque 3']]

In [91]:
[df_historic,requested_torques]= get_requested_torques(manv_ref)

In [94]:
print(requested_torques)

     Maniobra  Torque 1  Torque 2  Torque 3
554  NSSK0811 -0.142286 -0.012945  0.056209


In [40]:
average_torques

[0.0015112709446428551, -0.001275171920053749, 0.005914713588734078]

In [ ]:
#print(df.iloc[-1])

In [ ]:
requested_torques= df[df.Maniobra== manv_ref]
requested_torques[['Maniobra', 'Torque 1', 'Torque 2', 'Torque 3']]

In [ ]:
requested_torques.iloc[0][0]

## Another way to get info of an excel file

In [ ]:
import xlwings
SA=xlwings.Book("SKMTool_dummy.xlsm")
sht=xlwings.Sheet("Historico")
Epoch=sht.range("A25:d50").value
print(Epoch)
SA.close()

## Continue

In [ ]:
dir(requested_torques)

In [ ]:
requested_torques.values[0][1:4]

In [ ]:
average_torques[0]+float(requested_torques.values[0][1])

In [ ]:
new_values= average_torques[0]+requested_torques.iloc[0][1]
print(new_values)

In [ ]:
new_values=[average_torques[i]+requested_torques.iloc[0][i+1] for i in range(3)]

print(new_values)

## Suma

In [56]:
new_values= []
for i in range(3):
    #print(i)
    value= average_torques[i]+requested_torques.iloc[0][i+1]
    new_values.append(value)

#print(new_values)
new_values.insert(0, manv)
print(new_values)

    

['NSSK0812', -0.14077427694943861, -0.014219976805559776, 0.062123980702140473]


In [66]:
def get_new_values(manv, average_torques, requested_torques):
    new_values= []
    for i in range(3):
        #print(i)
        value= average_torques[i]+requested_torques.iloc[0][i+1]
        new_values.append(value)

    #print(new_values)
    new_values.insert(0, manv)
    return new_values
    

In [67]:
new_values= get_new_values(manv, average_torques, requested_torques)

In [57]:
answer= input("Quieres agregar estos nuevos valores al Historico del SKMTool?: \n" +str(new_values)[1:-1]+ "\n")

Quieres agregar estos nuevos valores al Historico del SKMTool?: 
'NSSK0812', -0.14077427694943861, -0.014219976805559776, 0.062123980702140473
n


In [103]:
import xlwings as xw

def make_a_decision(new_values, df_historic):
    answer= input("Quieres agregar estos nuevos valores al Historico del SKMTool?: (si/no)\n" +str(new_values)[1:-1]+ "\n")
    if answer.lower().replace(' ','') == 'si':
        #print('Dentro del if')
        new_row_data={'Maniobra':new_values[0],'Torque 1':new_values[1],'Torque 2':new_values[2],'Torque 3':new_values[3]}
        #print(new_row_data)
        df_new_values= pd.concat([df_historic, pd.DataFrame([new_row_data])], ignore_index= True)
        print(df_new_values)
    
        wb= xw.Book('SKMTool_dummy - Copy.xlsm')
        #wb= wb.books.active
        sheet= wb.sheets['Historico']
        sheet.range('A'+str(len(df_new_values)+1)).value= new_values[0]
        sheet.range('B'+str(len(df_new_values)+1)).value= new_values[1]
        sheet.range('C'+str(len(df_new_values)+1)).value= new_values[2]
        sheet.range('D'+str(len(df_new_values)+1)).value= new_values[3]
        #sheet.range('A'+str(len(df_new_values)+1)+":D"+str(len(df_new_values)+1)).value= new_values

        wb.save()
        wb.close()
        

In [104]:
make_a_decision(new_values, df_historic)

Quieres agregar estos nuevos valores al Historico del SKMTool?: (si/no)
'NSSK0812', -0.14077427694943861, -0.014219976805559776, 0.062123980702140473
si
     Maniobra  Torque 1  Torque 2  Torque 3 Unnamed: 4       Unnamed: 5  \
0     EWSK260  0.000800  0.026800 -0.000600        NaN              NaN   
1    NSSK0261 -0.133300 -0.008300 -0.026800        NaN              NaN   
2    EWSK0262  0.005000 -0.006000 -0.001000        NaN              NaN   
3    NSSK0263 -0.108300 -0.007200 -0.016600        NaN              NaN   
4    EWSK0264  0.000800  0.026800  0.000600        NaN              NaN   
5    NSSK0265 -0.103300 -0.007200 -0.018600        NaN              NaN   
6    NSSK0266 -0.098200  0.005400  0.049700        NaN              NaN   
7    EWSK0267 -0.001000  0.025300 -0.000300        NaN              NaN   
8    NSSK0268 -0.123000  0.003400  0.064700        NaN              NaN   
9    EWSK0269 -0.001000  0.021300 -0.000800        NaN              NaN   
10   NSSK0270 -0.12900

In [ ]:
#print(dir(answer))
print(answer.lower().replace(' ',''))

if answer == 'si':
    print('Dentro del if')

In [23]:
new_row_data={'Maniobra':new_values[0],'Torque 1':new_values[1],'Torque 2':new_values[2],'Torque 3':new_values[3]}
print(new_row_data)

{'Maniobra': 'NSSK0812', 'Torque 1': -0.14077427694943861, 'Torque 2': -0.014219976805559776, 'Torque 3': 0.062123980702140473}


In [98]:
df_new_values= pd.concat([df, pd.DataFrame([new_row_data])], ignore_index= True)
print(df_new_values)

NameError: name 'new_row_data' is not defined

In [ ]:
from openpyxl import load_workbook

try:
    book= load_workbook("SKMTool_dummy - Copy.xlsm")
except FileNotFoundError:
    print("File not found")

with pd.ExcelWriter("SKMTool_dummy - Copy.xlsm", engine="openpyxl", keep_vba= True) as writer:
    writer.book= book
    writer.vba_archive= book.vba_archive
    writer.sheets= dict((ws.title, ws) for ws in book.worksheets)
    df_new_values.to_excel(writer, sheet_name= "Historico", index=False)
    writer.close()

In [ ]:
with pd.ExcelWriter(
    "SKMTool_dummy - Copy.xlsm",
    engine="openpyxl",
    mode= "a",
    engine_kwargs={"keep_vba": True}
)as writer:
    df_new_values.to_excel(writer, sheet_name= "Historico")

In [ ]:
#import xlsxwriter, xlwings
import openpyxl
df_new_values.to_excel(excel_writer, 'SKMTool_dummy - Copy.xlsm', 'Historico', index=False)

In [ ]:
import xlsxwriter
import os

excel_file= 'test.xlsx'

if os.path.exists(excel_file):
    os.remove(excel_file)
    
workbook= xlsxwriter.Workbook(excel_file)
worksheet= workbook.add_worksheet()

worksheet.write('A1', 'Hello')
worksheet.write('B1', 'Puto')

workbook.close()

In [25]:
import xlwings as xw

wob= xw.Book('SKMTool_dummy - Copy.xlsm')

#wb= wb.books.active

sheet= wob.sheets['Historico']

sheet.range('A'+str(len(df_new_values)+1)).value= new_values[0]
sheet.range('B'+str(len(df_new_values)+1)).value= new_values[1]
sheet.range('C'+str(len(df_new_values)+1)).value= new_values[2]
sheet.range('D'+str(len(df_new_values)+1)).value= new_values[3]
#sheet.range('A'+str(len(df_new_values)+1)+":D"+str(len(df_new_values)+1)).value= new_values

wob.save()
wob.close()



In [ ]:
mx3_EWSK0812_PLAN.rpt #Plan (manv - a calcular)

IGNITION
CUTOFF
tHRUSTER CFG
Thrusters sec TOTAL



In [ ]:
SET DE IMPULSORES (LISTA)

SKMTool ... ATTITUDE_CONTROL_THRUSTER 
Las opciones para los thrusters de control son los siguientes

In [148]:
manv= "EWSK0812"
manvRef= "EWSK0810"

In [152]:
def get_file(option):
    if option == "EWSK0812":
        print(manv)
        #return manv
    elif option == "EWSK0810":
        print(manvRef)
        #return manvRef
        

In [153]:
get_file(manv)

EWSK0812


In [154]:
get_file(manvRef)

EWSK0810


In [155]:
respuesta=input("Test")
print(type(respuesta))

Test1
<class 'str'>


In [157]:
type(str(1))

str

In [162]:
a=[1,2,4]

for i in range(0, len(a)):
    print(a[i])

1
2
4


In [166]:
b=list(range(len(a)))
print(b)

[0, 1, 2]


In [221]:
ignition=[('2025', '10', '17', '13', '38', '10', '003')]
#print('-'.join(ignition[0]))
date_='-'.join(ignition[0])
print(date_)

2025-10-17-13-38-10-003


In [197]:
from datetime import datetime

print(dir(datetime))
print(datetime.now().weekday())

['__add__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__rsub__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', 'astimezone', 'combine', 'ctime', 'date', 'day', 'dst', 'fold', 'fromordinal', 'fromtimestamp', 'hour', 'isocalendar', 'isoformat', 'isoweekday', 'max', 'microsecond', 'min', 'minute', 'month', 'now', 'replace', 'resolution', 'second', 'strftime', 'strptime', 'time', 'timestamp', 'timetuple', 'timetz', 'today', 'toordinal', 'tzinfo', 'tzname', 'utcfromtimestamp', 'utcnow', 'utcoffset', 'utctimetuple', 'weekday', 'year']
3


In [226]:
my_date= datetime.strptime(date_, "%Y-%m-%d-%H-%M-%S-%f")
print(my_date)
print(my_date.strftime("%Y-%j-%H-%M-%S"))

2025-10-17 13:38:10.003000
2025-290-13-38-10


In [235]:
my_date= datetime(int(ignition[0][0]),int(ignition[0][1]),int(ignition[0][2]),int(ignition[0][3]),int(ignition[0][4]),int(ignition[0][5]))
print(my_date)
print(my_date.strftime("%Y-%j-%H-%M-%S"))

2025-10-17 13:38:10.003000
2025-290-13-38-10


In [231]:
ignition[0][1]

'10'

In [236]:
a=0.123456789

In [242]:
a.format("%.4f")

AttributeError: 'float' object has no attribute 'format'

In [246]:
format(a,".4f")

'0.1235'